In [65]:
import pandas as pd
import numpy as np
import torch
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from ast import literal_eval
import matplotlib.pyplot as plt
from textstat.textstat import textstat
from gensim.corpora import wikicorpus
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import nltk
from collections import defaultdict
%matplotlib inline
# Make it pretty
plt.style.use('ggplot')

In [66]:
file = '../data/enwiki.observations.text_wp10.30k.tsv'
raw_data = pd.read_csv(file, sep='\t', header=None)
data = pd.DataFrame(data=list(raw_data[0].apply(literal_eval)))

In [67]:
data = data[data['text'] != ""]
data = data[data['text'].str.contains("#redirect") == False]
data = data[data['text'].str.contains("may refer to:\n\n*") == False]
data = data[data['text'].str.contains("can refer to:\n") == False]
data = data[data['text'].str.contains("could refer to:\n") == False]
data = data[data['text'].str.contains("#REDIRECT") == False]
data = data[data['text'].str.contains("== Matches ==\n:") == False]
data = data[data['text'].str.contains("{{underconstruction") == False]

In [68]:
classes = {"stub": 0, "start": 1, "c": 2, "b": 3, "ga": 4, "fa": 5} 
data["label"] = data['label'].map(classes)

In [69]:
data = data[15000:].copy()

In [74]:
def clean_wiki_markup(raw_article):
    semi_cleaned_article = wikicorpus.filter_wiki(raw_article)
    cleaned_article = semi_cleaned_article.replace("\n", "").replace("\'", "").replace("()", "").replace("=", "").replace("|alt","").replace("\xa0","")
    return cleaned_article
def find_num_categories(raw_article):
    return raw_article.count("[[Category:")
def find_num_images(raw_article):
    return raw_article.count("[[Image:")
def find_num_ISBN(raw_article):
    return raw_article.count("ISBN")
def find_num_references(raw_article):
    return raw_article.count("</ref>")
def find_article_length(cleaned_article):
    return len(cleaned_article)
def find_num_difficult_words(cleaned_article):
    return textstat.difficult_words(cleaned_article)
def find_dale_chall_readability_score(cleaned_article):
    return textstat.dale_chall_readability_score(cleaned_article)
def find_automated_readability_index(cleaned_article):
    return textstat.automated_readability_index(cleaned_article)
def find_linsear_write_formula(cleaned_article):
    return textstat.linsear_write_formula(cleaned_article)
def find_gunning_fog_index(cleaned_article):
    return textstat.gunning_fog(cleaned_article)
def find_syllable_count(cleaned_article):
    return textstat.syllable_count(cleaned_article)
def find_lexicon_count(cleaned_article):
    return textstat.lexicon_count(cleaned_article, removepunct=True)
def find_sentence_count(cleaned_article):
    return textstat.sentence_count(cleaned_article)
def find_smog_index(cleaned_article):
    return textstat.smog_index(cleaned_article)
def find_num_web_citations(raw_article):
    return raw_article.count("{{cite web")
def find_num_book_citations(raw_article):
    return raw_article.count("{{cite book")
def find_num_news_citations(raw_article):
    return raw_article.count("{{cite news")
def find_num_quotes(raw_article):
    return raw_article.count("quote=")
def find_num_h3_headers(raw_article):
    return raw_article.count("\n===")
def find_num_internal_links(raw_article):
    return (raw_article.count("[[") // 2)
def find_num_h2_headers(raw_article):
    return (raw_article.count("\n==") - find_num_h3_headers(raw_article))
def find_num_note_tags(raw_article):
    return raw_article.count("{{note")
def find_num_bullet_points(raw_article):
    return (raw_article.count("*"))
def find_num_underlines(raw_article):
    return (raw_article.count("<u>"))
def find_num_journal_citations(raw_article):
    return (raw_article.count("{{cite journal"))
def find_num_about_links(raw_article):
    return (raw_article.count("{{About"))
def find_num_wikitables(raw_article):
    return (raw_article.count('class="wikitable'))
def find_num_footnotes(raw_article):
    return raw_article.count("{{")
def find_infobox(raw_article):
    return int('{{Infobox' in raw_article)

In [75]:
def create_engineered_features(raw_text):
    cleaned_text = clean_wiki_markup(raw_text)
    return {
        'cleaned_text': cleaned_text,
        'num_web_citations': find_num_web_citations(raw_text),
        'num_book_citations': find_num_book_citations(raw_text),
        'num_news_citations': find_num_news_citations(raw_text),
        'num_quotes': find_num_quotes(raw_text),
        'num_h3_headers': find_num_h3_headers(raw_text),
        'num_internal_links': find_num_internal_links(raw_text),
        'num_h2_headers': find_num_h2_headers(raw_text),
        'has_infobox': find_infobox(raw_text),
        'num_categories': find_num_categories(raw_text),
        'num_images': find_num_images(raw_text),
        'num_ISBN': find_num_ISBN(raw_text),
        'num_references': find_num_references(raw_text),
        'article_length': find_article_length(raw_text),
        'num_difficult_words': find_num_difficult_words(cleaned_text),
        'dale_chall_readability_score': find_dale_chall_readability_score(cleaned_text),
        'readability_index': find_automated_readability_index(cleaned_text),
        'linsear_write_formula': find_linsear_write_formula(cleaned_text),
        'gunning_fog_index': find_gunning_fog_index(cleaned_text),
        'smog_index': find_smog_index(cleaned_text),
        'syllable_count': find_syllable_count(cleaned_text),
        'lexicon_count': find_lexicon_count(cleaned_text),
        'sentence_count': find_sentence_count(cleaned_text),
        'num_footnotes': find_num_footnotes(raw_text),
        'num_note_tags': find_num_note_tags(raw_text),
        'num_underlines': find_num_underlines(raw_text),
        'num_journal_citations': find_num_journal_citations(raw_text),
        'num_about_links': find_num_about_links(raw_text),
        'num_wikitables': find_num_wikitables(raw_text)}

def get_engineered_dataframe_yes_label(raw_dataframe):
    engineered_df = pd.DataFrame(raw_dataframe['text'].apply(create_engineered_features).tolist())
    engineered_df['label'] = raw_dataframe['label']
    return engineered_df

In [76]:
engineered_df = get_engineered_dataframe_yes_label(data)

Error(DCRS): Word Count is zero cannot divide
Error(ARI) : Sentence count is zero, cannot divide
Error(GF): Word Count is Zero, cannot divide
Error(DCRS): Word Count is zero cannot divide
Error(ARI) : Sentence count is zero, cannot divide
Error(GF): Word Count is Zero, cannot divide


In [8]:
data['cleaned_text'] = data['text'].apply(clean_wiki_markup)
data['num_web_citations'] = data['text'].apply(find_num_web_citations)
data['num_book_citations'] = data['text'].apply(find_num_book_citations)
data['num_news_citations'] = data['text'].apply(find_num_news_citations)
data['num_quotes'] = data['text'].apply(find_num_quotes)
data['num_h3_headers'] = data['text'].apply(find_num_h3_headers)
data['num_internal_links'] = data['text'].apply(find_num_internal_links)
data['num_h2_headers'] = data['text'].apply(find_num_h2_headers)
data['has_infobox'] = data['text'].str.contains('{{Infobox').astype(int)
data['num_categories'] = data['text'].apply(find_num_categories)
data['num_images'] = data['text'].apply(find_num_images)
data['num_ISBN'] = data['text'].apply(find_num_ISBN)
data['num_references'] = data['text'].apply(find_num_references)
data['article_length'] = data['text'].apply(find_article_length)
data['num_difficult_words'] = data['cleaned_text'].apply(find_num_difficult_words)
data['dale_chall_readability_score'] = data['cleaned_text'].apply(find_dale_chall_readability_score)
data['readability_index'] = data['cleaned_text'].apply(find_automated_readability_index)
data['linsear_write_formula'] = data['cleaned_text'].apply(find_linsear_write_formula)
data['gunning_fog_index'] = data['cleaned_text'].apply(find_gunning_fog_index)
data['smog_index'] = data['cleaned_text'].apply(find_smog_index)
data['syllable_count'] = data['cleaned_text'].apply(find_syllable_count)
data['lexicon_count'] = data['cleaned_text'].apply(find_lexicon_count)
data['sentence_count'] = data['cleaned_text'].apply(find_sentence_count)
data['num_footnotes'] = data['text'].apply(find_num_footnotes)
data['num_note_tags'] = data['text'].apply(find_num_note_tags)
data['num_underlines'] = data['text'].apply(find_num_underlines)
data['num_journal_citations'] = data['text'].apply(find_num_journal_citations)
data['num_about_links'] = data['text'].apply(find_num_about_links)
data['num_wikitables'] = data['text'].apply(find_num_wikitables)

Error(DCRS): Word Count is zero cannot divide
Error(DCRS): Word Count is zero cannot divide
Error(ARI) : Sentence count is zero, cannot divide
Error(ARI) : Sentence count is zero, cannot divide
Error(GF): Word Count is Zero, cannot divide
Error(GF): Word Count is Zero, cannot divide


In [77]:
engineered_df.dropna(inplace=True)

In [79]:
engineered_df

,article_length,cleaned_text,dale_chall_readability_score,gunning_fog_index,has_infobox,lexicon_count,linsear_write_formula,num_ISBN,num_about_links,num_book_citations,...,num_quotes,num_references,num_underlines,num_web_citations,num_wikitables,readability_index,sentence_count,smog_index,syllable_count,label


In [24]:
X = data.loc[:, ['has_infobox','num_categories','num_images','num_ISBN','num_references','article_length',
                'num_difficult_words','dale_chall_readability_score','readability_index','linsear_write_formula',
                'gunning_fog_index', 'num_web_citations','num_book_citations','num_news_citations',
                'num_quotes','num_h3_headers','num_internal_links', 'num_h2_headers', 'syllable_count',
                'lexicon_count', 'sentence_count','num_footnotes', 'num_note_tags', 'num_underlines', 'num_journal_citations',
                'num_about_links', 'num_wikitables', 'smog_index']]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.20, random_state=910)

In [51]:
import xgboost as xgb

xbg_model = xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.01, max_depth=10, alpha = 10, n_estimators = 500)

In [52]:
xbg_model.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [53]:
preds = xbg_model.predict(X_test)

In [56]:
mse = (mean_squared_error(y_test, preds))

print(f"MSE: {mse}")

RMSE: 0.7371038304247567


In [58]:
rf = RandomForestRegressor(n_estimators=500, random_state=910)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
print(mean_squared_error(y_test, predictions))

0.7322181156009326


In [60]:
import pickle
pkl_filename = "random_forest_aug22.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(rf, file)

In [61]:
pkl_filename2 = "XGB_aug22.pkl"  
with open(pkl_filename2, 'wb') as file:  
    pickle.dump(xbg_model, file)

XGB_aug22.pkl                    random_forest_aug22.pkl
gradient_boosted_ensemble.ipynb  random_forest_model.py
hash_vectorizer_model.py         random_forest_test_pipeline.py
hash_vectorizer_test_pipeline.py word_embedding_model.ipynb
neural_network_model.ipynb
